# Анализ лояльности пользователей Яндекс Афиши

Целью исследования является изучить поведение пользователей в сервисе. Для этого необходимо провести исследовательский анализ данных, чтобы понять, какие пользователи с большей вероятностью возвращаются на платформу и делают заказы повторно.

## Этапы выполнения проекта

### 1. Загрузка данных и их предобработка

---

**Задача 1.1:** Напишите SQL-запрос, выгружающий в датафрейм pandas необходимые данные. Используйте следующие параметры для подключения к базе данных `data-analyst-afisha`:

- **Хост** — `rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net`
- **База данных** — `data-analyst-afisha`
- **Порт** — `6432`
- **Аутентификация** — `Database Native`
- **Пользователь** — `praktikum_student`
- **Пароль** — `Sdf4$2;d-d30pp`

Для выгрузки используйте запрос из предыдущего урока и библиотеку SQLAlchemy.

Выгрузка из базы данных SQL должна позволить собрать следующие данные:

- `user_id` — уникальный идентификатор пользователя, совершившего заказ;
- `device_type_canonical` — тип устройства, с которого был оформлен заказ (`mobile` — мобильные устройства, `desktop` — стационарные);
- `order_id` — уникальный идентификатор заказа;
- `order_dt` — дата создания заказа (используйте данные `created_dt_msk`);
- `order_ts` — дата и время создания заказа (используйте данные `created_ts_msk`);
- `currency_code` — валюта оплаты;
- `revenue` — выручка от заказа;
- `tickets_count` — количество купленных билетов;
- `days_since_prev` — количество дней от предыдущей покупки пользователя, для пользователей с одной покупкой — значение пропущено;
- `event_id` — уникальный идентификатор мероприятия;
- `service_name` — название билетного оператора;
- `event_type_main` — основной тип мероприятия (театральная постановка, концерт и так далее);
- `region_name` — название региона, в котором прошло мероприятие;
- `city_name` — название города, в котором прошло мероприятие.

---


In [ ]:
# Используйте ячейки типа Code для вашего кода,
# а ячейки типа Markdown для комментариев и выводов

In [ ]:
# При необходимости добавляйте новые ячейки для кода или текста

In [ ]:
import os
from dotenv import load_dotenv

import pandas as pd
from sqlalchemy import create_engine 
import matplotlib.pyplot as plt
import seaborn as sns
from phik import phik_matrix

In [ ]:
load_dotenv()

db_config = {'user': os.environ.get('DB_USER'), # имя пользователя
             'pwd': os.environ.get('DB_PASSWORD'), # пароль
             'host': os.environ.get('DB_HOST'),
             'port': os.environ.get('DB_PORT'), # порт подключения
             'db': os.environ.get('DB_NAME') # название базы данных
             } 

In [ ]:
query = """
SELECT
    p.user_id,
    p.device_type_canonical,
    p.order_id,
    p.created_dt_msk AS order_dt,
    p.created_ts_msk AS order_ts,
    p.currency_code,
    p.revenue,
    p.tickets_count,
    ROUND(
        EXTRACT(DAY FROM (
            p.created_dt_msk - LAG(p.created_dt_msk) OVER (PARTITION BY p.user_id ORDER BY p.created_dt_msk)))::numeric, 2
    ) AS days_since_prev,
    p.event_id,
    p.service_name,
    e.event_type_main,
    r.region_name,
    c.city_name
FROM afisha.purchases p
INNER JOIN afisha.events e ON p.event_id = e.event_id
JOIN afisha.city c ON e.city_id = c.city_id
JOIN afisha.regions r ON c.region_id = r.region_id
WHERE
    p.device_type_canonical IN ('mobile', 'desktop')
    AND e.event_type_main != 'фильм'
ORDER BY
    p.user_id ASC,
    p.created_dt_msk ASC
"""

In [ ]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
) 
engine = create_engine(connection_string)

In [ ]:
df = pd.read_sql_query(query, con=engine)

In [ ]:
df.head()

---

**Задача 1.2:** Изучите общую информацию о выгруженных данных. Оцените корректность выгрузки и объём полученных данных.

Предположите, какие шаги необходимо сделать на стадии предобработки данных — например, скорректировать типы данных.

Зафиксируйте основную информацию о данных в кратком промежуточном выводе.

---

In [ ]:
df.info()

В нашем датафрейме 290611 строк. Пропуски есть лишь в одном столбце - days_since_prev. Они получаются у тех пользователей,
у которых нет предыдыщей покупки

---

###  2. Предобработка данных

Выполните все стандартные действия по предобработке данных:

---

**Задача 2.1:** Данные о выручке сервиса представлены в российских рублях и казахстанских тенге. Приведите выручку к единой валюте — российскому рублю.

Для этого используйте датасет с информацией о курсе казахстанского тенге по отношению к российскому рублю за 2024 год — `final_tickets_tenge_df.csv`. Его можно загрузить по пути `https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv')`

Значения в рублях представлено для 100 тенге.

Результаты преобразования сохраните в новый столбец `revenue_rub`.

---


In [ ]:
rev_df = pd.read_csv('https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv')

In [ ]:
rev_df.head()

In [ ]:
rev_df.info()

In [ ]:
rev_df['data'] = pd.to_datetime(rev_df['data'])
rev_df = rev_df[['data', 'curs']]
rev_df

In [ ]:
df = df.merge(rev_df, left_on='order_dt', right_on='data', how='left')
df.head()

In [ ]:
def func(row):
    if row['currency_code'] == 'rub':
        return row['revenue']
    return row['revenue'] * (row['curs'] / 100)
        

df['revenue_rub'] = df.apply(func, axis = 1)

In [ ]:
df = df.drop(['data', 'curs'], axis=1)

In [ ]:
df.head()

---

**Задача 2.2:**

- Проверьте данные на пропущенные значения. Если выгрузка из SQL была успешной, то пропуски должны быть только в столбце `days_since_prev`.
- Преобразуйте типы данных в некоторых столбцах, если это необходимо. Обратите внимание на данные с датой и временем, а также на числовые данные, размерность которых можно сократить.
- Изучите значения в ключевых столбцах. Обработайте ошибки, если обнаружите их.
    - Проверьте, какие категории указаны в столбцах с номинальными данными. Есть ли среди категорий такие, что обозначают пропуски в данных или отсутствие информации? Проведите нормализацию данных, если это необходимо.
    - Проверьте распределение численных данных и наличие в них выбросов. Для этого используйте статистические показатели, гистограммы распределения значений или диаграммы размаха.
        
        Важные показатели в рамках поставленной задачи — это выручка с заказа (`revenue_rub`) и количество билетов в заказе (`tickets_count`), поэтому в первую очередь проверьте данные в этих столбцах.
        
        Если обнаружите выбросы в поле `revenue_rub`, то отфильтруйте значения по 99 перцентилю.

После предобработки проверьте, были ли отфильтрованы данные. Если были, то оцените, в каком объёме. Сформулируйте промежуточный вывод, зафиксировав основные действия и описания новых столбцов.

---

In [ ]:
df.isna().sum()

In [ ]:
df.isna().mean()

In [ ]:
df['device_type_canonical'].value_counts()

In [ ]:
df['service_name'].value_counts()

In [ ]:
df['event_type_main'].value_counts()

In [ ]:
df.duplicated().sum(), df.duplicated(subset=['user_id','order_id','event_id']).sum()

In [ ]:
df[['revenue_rub','tickets_count']].describe()

In [ ]:
plt.figure(figsize=(10,6))
df.boxplot('tickets_count', vert=False)
plt.title('Количество билетов в заказе')
plt.ylabel('Билеты')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
df.boxplot('revenue_rub', vert=False)
plt.title('Выручка с заказа')
plt.ylabel('Выручка')
plt.show()

In [ ]:
per_99 = df['revenue_rub'].quantile(0.99)

In [ ]:
dff = df[(df['revenue_rub'] <= per_99) & (df['revenue_rub'] > 0)].copy()

In [ ]:
dff.head()

In [ ]:
dff.info()

In [ ]:
dff[['revenue_rub','tickets_count']].describe()

In [ ]:
plt.figure(figsize=(10,6))
dff.boxplot('revenue_rub', vert=False)
plt.title('Выручка с заказа')
plt.ylabel('Выручка')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
df.boxplot('tickets_count', vert=False)
plt.title('Количество билетов в заказе')
plt.ylabel('Билеты')
plt.show()

Типы данных оставляем как есть, поскольку объем маленький
Пропуски есть в столбце days_since_prev, но они обусловлены тем, что пользователь сделал всего один заказ
Явных и неявных дубликтов не выявлено

In [ ]:
dff.duplicated().sum(), dff.duplicated(subset=['user_id','order_id','event_id']).sum()

Дубликаты не обнаружены

In [ ]:
for column in ['service_name', 'event_type_main', 'region_name']:
    print(f"\nColumn: {column}")
    print("Unique values:", df[column].unique())
    print("Missing values count:", df[column].isna().sum())
    print("Value counts:")
    print(df[column].value_counts(dropna=False), "\n")  

In [ ]:
df.shape[0]

In [ ]:
dff.shape[0]

In [ ]:
dff.shape[0] / df.shape[0]

---

### 3. Создание профиля пользователя

В будущем отдел маркетинга планирует создать модель для прогнозирования возврата пользователей. Поэтому сейчас они просят вас построить агрегированные признаки, описывающие поведение и профиль каждого пользователя.

---

**Задача 3.1.** Постройте профиль пользователя — для каждого пользователя найдите:

- дату первого и последнего заказа;
- устройство, с которого был сделан первый заказ;
- регион, в котором был сделан первый заказ;
- билетного партнёра, к которому обращались при первом заказе;
- жанр первого посещённого мероприятия (используйте поле `event_type_main`);
- общее количество заказов;
- средняя выручка с одного заказа в рублях;
- среднее количество билетов в заказе;
- среднее время между заказами.

После этого добавьте два бинарных признака:

- `is_two` — совершил ли пользователь 2 и более заказа;
- `is_five` — совершил ли пользователь 5 и более заказов.

**Рекомендация:** перед тем как строить профиль, отсортируйте данные по времени совершения заказа.

---


In [ ]:
dff = dff.sort_values(by='order_ts').copy()

In [ ]:
dff.head()

In [ ]:
profile = dff.sort_values(by='order_ts').groupby('user_id').agg(
    first_order_date=('order_dt', 'first'),
    last_order_date=('order_dt', 'last'),
    first_device=('device_type_canonical', 'first'),
    first_region=('region_name', 'first'),
    first_service=('service_name', 'first'),
    first_event_type=('event_type_main', 'first'),
    total_orders=('order_id', 'count'),
    avg_revenue_rub=('revenue_rub', 'mean'),
    avg_tickets=('tickets_count', 'mean'),
    avg_days_between=('days_since_prev', 'mean')
).reset_index()

In [ ]:
profile.head()

In [ ]:
profile['is_two'] = profile.apply(lambda row: row['total_orders'] >= 2, axis=1)

In [ ]:
profile['is_five'] = profile.apply(lambda row: row['total_orders'] >= 5, axis=1)

In [ ]:
profile.head()

---

**Задача 3.2.** Прежде чем проводить исследовательский анализ данных и делать выводы, важно понять, с какими данными вы работаете: насколько они репрезентативны и нет ли в них аномалий.

Используя данные о профилях пользователей, рассчитайте:

- общее число пользователей в выборке;
- среднюю выручку с одного заказа;
- долю пользователей, совершивших 2 и более заказа;
- долю пользователей, совершивших 5 и более заказов.

Также изучите статистические показатели:

- по общему числу заказов;
- по среднему числу билетов в заказе;
- по среднему количеству дней между покупками.

По результатам оцените данные: достаточно ли их по объёму, есть ли аномальные значения в данных о количестве заказов и среднем количестве билетов?

Если вы найдёте аномальные значения, опишите их и примите обоснованное решение о том, как с ними поступить:

- Оставить и учитывать их при анализе?
- Отфильтровать данные по какому-то значению, например, по 95-му или 99-му перцентилю?

Если вы проведёте фильтрацию, то вычислите объём отфильтрованных данных и выведите статистические показатели по обновлённому датасету.

In [ ]:
profile['user_id'].count()

In [ ]:
profile['avg_revenue_rub'].mean()

In [ ]:
profile['is_two'].mean()

In [ ]:
profile['is_five'].mean()

In [ ]:
profile[['total_orders','avg_tickets','avg_days_between']].describe()

In [ ]:
profile.head()

In [ ]:
profile[(profile['total_orders'] <= profile['total_orders'].quantile(0.99))].describe()

In [ ]:
profile[(profile['total_orders'] <= profile['total_orders'].quantile(0.95))].describe()

In [ ]:
pr_f = profile[(profile['total_orders'] <= profile['total_orders'].quantile(0.95))].copy()

In [ ]:
pr_f.describe()

Аномальное число заказов - 9987
Фильтруем по 95 перцентилю, поскольку в данном случае распределению будет более равномерным

---

### 4. Исследовательский анализ данных

Следующий этап — исследование признаков, влияющих на возврат пользователей, то есть на совершение повторного заказа. Для этого используйте профили пользователей.



#### 4.1. Исследование признаков первого заказа и их связи с возвращением на платформу

Исследуйте признаки, описывающие первый заказ пользователя, и выясните, влияют ли они на вероятность возвращения пользователя.

---

**Задача 4.1.1.** Изучите распределение пользователей по признакам.

- Сгруппируйте пользователей:
    - по типу их первого мероприятия;
    - по типу устройства, с которого совершена первая покупка;
    - по региону проведения мероприятия из первого заказа;
    - по билетному оператору, продавшему билеты на первый заказ.
- Подсчитайте общее количество пользователей в каждом сегменте и их долю в разрезе каждого признака. Сегмент — это группа пользователей, объединённых определённым признаком, то есть объединённые принадлежностью к категории. Например, все клиенты, сделавшие первый заказ с мобильного телефона, — это сегмент.
- Ответьте на вопрос: равномерно ли распределены пользователи по сегментам или есть выраженные «точки входа» — сегменты с наибольшим числом пользователей?

---


In [ ]:
pr_f.head()

In [ ]:
pr_f.groupby('first_event_type').agg(
    user_count=('user_id', 'count'), 
    user_percentage=('user_id', lambda x: len(x) / pr_f.shape[0])
).sort_values(by = 'user_count', ascending=False)

In [ ]:
pr_f.groupby('first_device').agg(
    user_count=('user_id', 'count'), 
    user_percentage=('user_id', lambda x: len(x) / pr_f.shape[0])
).sort_values(by = 'user_count', ascending=False)

In [ ]:
pr_f.groupby('first_region').agg(
    user_count=('user_id', 'count'), 
    user_percentage=('user_id', lambda x: len(x) / pr_f.shape[0])
).sort_values(by = 'user_count', ascending=False)

In [ ]:
pr_f.groupby('first_service').agg(
    user_count=('user_id', 'count'), 
    user_percentage=('user_id', lambda x: len(x) / pr_f.shape[0])
).sort_values(by = 'user_count', ascending=False)

Данные распределены неравновмерно
По типу лидируют концерты 44%
Большинство заказаов - 80% с мобильных устройств
Регион - Каменевский регион
Лидер билетных операторов - Билеты без проблем

---

**Задача 4.1.2.** Проанализируйте возвраты пользователей:

- Для каждого сегмента вычислите долю пользователей, совершивших два и более заказа.
- Визуализируйте результат подходящим графиком. Если сегментов слишком много, то поместите на график только 10 сегментов с наибольшим количеством пользователей. Такое возможно с сегментами по региону и по билетному оператору.
- Ответьте на вопросы:
    - Какие сегменты пользователей чаще возвращаются на Яндекс Афишу?
    - Наблюдаются ли успешные «точки входа» — такие сегменты, в которых пользователи чаще совершают повторный заказ, чем в среднем по выборке?

При интерпретации результатов учитывайте размер сегментов: если в сегменте мало пользователей (например, десятки), то доли могут быть нестабильными и недостоверными, то есть показывать широкую вариацию значений.

---


In [ ]:
ev = pr_f.groupby('first_event_type').agg({'is_two':'mean'}).sort_values(by = 'is_two', ascending=False)
ev

In [ ]:
ev.plot(kind='bar', grid=True,
        title='Доля пользователей, совершивших два и более заказа по мероприятию', xlabel='Мероприятие', ylabel='Доля')

In [ ]:
dv = pr_f.groupby('first_device').agg({'is_two':'mean'}).sort_values(by = 'is_two', ascending=False)
dv

In [ ]:
dv.plot(kind='bar', grid=True,
        title='Доля пользователей, совершивших два и более заказа по устройству', xlabel='Устройство', ylabel='Доля')

In [ ]:
rv = pr_f.groupby('first_region').agg({'is_two':'mean'}).sort_values(by = 'is_two', ascending=False)
rv

In [ ]:
rv.iloc[:10].plot(kind='bar', grid=True,
        title='Доля пользователей, совершивших два и более заказа по региону', xlabel='Регион', ylabel='Доля')

In [ ]:
ov = pr_f.groupby('first_service').agg({'is_two':'mean'}).sort_values(by = 'is_two', ascending=False)
ov

In [ ]:
ov.iloc[:10].plot(kind='bar', grid=True,
        title='Доля пользователей, совершивших два и более заказа по оператору', xlabel='Оператор', ylabel='Доля')

Распределение по типу мероприятия и типу устройства относительно равномерно
Для исследования региона были оставлены 10 популряных значений, в них распределение тоже равномерно за исключением первого значения равного 1
Аналогичная ситуация и с операторами, распределение в целом равномерно, за исключением первого значения


---

**Задача 4.1.3.** Опираясь на выводы из задач выше, проверьте продуктовые гипотезы:

- **Гипотеза 1.** Тип мероприятия влияет на вероятность возврата на Яндекс Афишу: пользователи, которые совершили первый заказ на спортивные мероприятия, совершают повторный заказ чаще, чем пользователи, оформившие свой первый заказ на концерты.
- **Гипотеза 2.** В регионах, где больше всего пользователей посещают мероприятия, выше доля повторных заказов, чем в менее активных регионах.

---

In [ ]:
pr_f.groupby('first_event_type').agg({'is_two':'mean'}).sort_values(by = 'is_two', ascending=False)

Гипотеза не подтверждается, пользователи совершившие первый заказал на спортивные мероприятия совершают потворный заказ реже,
чем пользователи на концерты

In [ ]:
pr_f.groupby('first_region').agg({'is_two':'mean'}).sort_values(by = 'first_region', ascending=True)

Гипотеза подтверждается, в регионах, где больше всего пользователей посещают мероприятия, 
выше доля повторных заказов, чем в менее активных регионах

---

#### 4.2. Исследование поведения пользователей через показатели выручки и состава заказа

Изучите количественные характеристики заказов пользователей, чтобы узнать среднюю выручку сервиса с заказа и количество билетов, которое пользователи обычно покупают.

Эти метрики важны не только для оценки выручки, но и для оценки вовлечённости пользователей. Возможно, пользователи с более крупными и дорогими заказами более заинтересованы в сервисе и поэтому чаще возвращаются.

---

**Задача 4.2.1.** Проследите связь между средней выручкой сервиса с заказа и повторными заказами.

- Постройте сравнительные гистограммы распределения средней выручки с билета (`avg_revenue_rub`):
    - для пользователей, совершивших один заказ;
    - для вернувшихся пользователей, совершивших 2 и более заказа.
- Ответьте на вопросы:
    - В каких диапазонах средней выручки концентрируются пользователи из каждой группы?
    - Есть ли различия между группами?

Текст на сером фоне:
    
**Рекомендация:**

1. Используйте одинаковые интервалы (`bins`) и прозрачность (`alpha`), чтобы визуально сопоставить распределения.
2. Задайте параметру `density` значение `True`, чтобы сравнивать форму распределений, даже если число пользователей в группах отличается.

---


In [ ]:
# plt.figure(figsize=(10,6))
# pr_f[pr_f['is_two'] == False].hist(column='avg_revenue_rub', bins=15, alpha=1, density=True)

# plt.title('Распределение средней выручки с билета (1 заказ)')
# plt.xlabel('Выручка')
# plt.ylabel('Распределение')
# plt.show()

In [ ]:
# plt.figure(figsize=(10,6))
# pr_f[pr_f['is_two'] == True].hist(column='avg_revenue_rub', bins=15, alpha=1, density=True)

# plt.title('Распределение средней выручки с билета (более 2 заказаов)')
# plt.xlabel('Выручка')
# plt.ylabel('Распределение')
# plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))


pr_f[pr_f['is_two'] == False]['avg_revenue_rub'].hist(
    bins=15, 
    alpha=0.6, 
    density=True,
    color='blue',
    label='1 заказ',
    ax=ax
)

pr_f[pr_f['is_two'] == True]['avg_revenue_rub'].hist(
    bins=15, 
    alpha=0.6, 
    density=True,
    color='red',
    label='≥2 заказов',
    ax=ax
)

ax.set_title('Сравнение распределений средней выручки с билета', fontsize=14)
ax.set_xlabel('Выручка, руб', fontsize=12)
ax.set_ylabel('Плотность распределения', fontsize=12)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Основные значения сосредоточены около 500р
Между группами имеется небольшое отличие. У пользователей покупающих больше 2 билетов выручка несколько больше

---

**Задача 4.2.2.** Сравните распределение по средней выручке с заказа в двух группах пользователей:

- совершившие 2–4 заказа;
- совершившие 5 и более заказов.

Ответьте на вопрос: есть ли различия по значению средней выручки с заказа между пользователями этих двух групп?

---


In [ ]:
# plt.figure(figsize=(10,6))
# pr_f[(pr_f['is_two'] == True) & (pr_f['is_five'] == False)].hist(column='avg_revenue_rub', bins=15, alpha=1, density=True)

# plt.title('Распределение средней выручки с билета (2 - 4 заказа)')
# plt.xlabel('Выручка')
# plt.ylabel('Распределение')
# plt.show()

In [ ]:
# plt.figure(figsize=(10,6))
# pr_f[pr_f['is_five'] == True].hist(column='avg_revenue_rub', bins=15, alpha=1, density=True)

# plt.title('Распределение средней выручки с билета (более 5 заказов)')
# plt.xlabel('Выручка')
# plt.ylabel('Распределение')
# plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))


pr_f[(pr_f['is_two'] == True) & (pr_f['is_five'] == False)]['avg_revenue_rub'].hist(
    bins=15, 
    alpha=0.6, 
    density=True,
    color='blue',
    label='2-4 заказа',
    ax=ax
)


pr_f[pr_f['is_five'] == True]['avg_revenue_rub'].hist(
    bins=15, 
    alpha=0.6, 
    density=True,
    color='red',
    label='≥5 заказов',
    ax=ax
)


ax.set_title('Сравнение распределений средней выручки с билета\nпо количеству заказов', fontsize=14)
ax.set_xlabel('Выручка, руб', fontsize=12)
ax.set_ylabel('Плотность распределения', fontsize=12)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Немного большая средняя выручка наблюдается у группы пользователей, совершивших более 5 заказов

---

**Задача 4.2.3.** Проанализируйте влияние среднего количества билетов в заказе на вероятность повторной покупки.

- Изучите распределение пользователей по среднему количеству билетов в заказе (`avg_tickets_count`) и опишите основные наблюдения.
- Разделите пользователей на несколько сегментов по среднему количеству билетов в заказе:
    - от 1 до 2 билетов;
    - от 2 до 3 билетов;
    - от 3 до 5 билетов;
    - от 5 и более билетов.
- Для каждого сегмента подсчитайте общее число пользователей и долю пользователей, совершивших повторные заказы.
- Ответьте на вопросы:
    - Как распределены пользователи по сегментам — равномерно или сконцентрировано?
    - Есть ли сегменты с аномально высокой или низкой долей повторных покупок?

---

In [ ]:
plt.figure(figsize=(10,6))
pr_f.hist(column='avg_tickets', bins=10)

plt.title('Распределение пользователей по среднему количеству билетов')
plt.xlabel('Билеты')
plt.ylabel('Распределение')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
pr_f.query('avg_tickets > 1 and avg_tickets < 2').hist(column='avg_tickets', bins=5)

plt.title('Распределение от 1 до 2 билетов')
plt.xlabel('Билеты')
plt.ylabel('Распределение')
plt.show()

In [ ]:
pr_f.query('avg_tickets > 1 and avg_tickets < 2').shape[0], pr_f.query('avg_tickets > 1 and avg_tickets < 2').shape[0] / pr_f.shape[0]

In [ ]:
plt.figure(figsize=(10,6))
pr_f.query('avg_tickets > 2 and avg_tickets < 3').hist(column='avg_tickets', bins=5)

plt.title('Распределение от 2 до 3 билетов')
plt.xlabel('Билеты')
plt.ylabel('Распределение')
plt.show()

In [ ]:
pr_f.query('avg_tickets > 2 and avg_tickets < 3').shape[0], pr_f.query('avg_tickets > 2 and avg_tickets < 3').shape[0] / pr_f.shape[0]

In [ ]:
plt.figure(figsize=(10,6))
pr_f.query('avg_tickets > 3 and avg_tickets < 5').hist(column='avg_tickets', bins=5)

plt.title('Распределение от 3 до 5 билетов')
plt.xlabel('Билеты')
plt.ylabel('Распределение')
plt.show()

In [ ]:
pr_f.query('avg_tickets > 3 and avg_tickets < 5').shape[0], pr_f.query('avg_tickets > 3 and avg_tickets < 5').shape[0] / pr_f.shape[0]

In [ ]:
plt.figure(figsize=(10,6))
pr_f.query('avg_tickets > 5').hist(column='avg_tickets', bins=5)

plt.title('Распределение от 5 билетов')
plt.xlabel('Билеты')
plt.ylabel('Распределение')
plt.show()

In [ ]:
pr_f.query('avg_tickets > 5').shape[0], pr_f.query('avg_tickets >= 5').shape[0] / pr_f.shape[0]

Чаще всего у пользователей примерно от 2 до 4 билетов

---

#### 4.3. Исследование временных характеристик первого заказа и их влияния на повторные покупки

Изучите временные параметры, связанные с первым заказом пользователей:

- день недели первой покупки;
- время с момента первой покупки — лайфтайм;
- средний интервал между покупками пользователей с повторными заказами.

---

**Задача 4.3.1.** Проанализируйте, как день недели, в которой была совершена первая покупка, влияет на поведение пользователей.

- По данным даты первого заказа выделите день недели.
- Для каждого дня недели подсчитайте общее число пользователей и долю пользователей, совершивших повторные заказы. Результаты визуализируйте.
- Ответьте на вопрос: влияет ли день недели, в которую совершена первая покупка, на вероятность возврата клиента?

---


In [ ]:
pr_f['day'] = pr_f['first_order_date'].dt.day_name()

In [ ]:
pr_f.head()

In [ ]:
pr_f['day'].value_counts()

In [ ]:
plt.figure(figsize=(10,6))
pr_f['day'].value_counts().plot(kind='bar', grid=True)
plt.title('Число пользователей по дням недели')
plt.xlabel('День')
plt.ylabel('Пользователи')
plt.show()

In [ ]:
pr_f['day'].value_counts(normalize=True)

In [ ]:
plt.figure(figsize=(10,6))
pr_f['day'].value_counts(normalize=True).plot(kind='bar', grid=True)
plt.title('Число пользователей по дням недели')
plt.xlabel('День')
plt.ylabel('Доля пользователи')
plt.show()

В целом распределение в течении недели равномерно. Однако можно заметить небольшой спад в понедельник и воскресенье.
Возможно это связано с рабочим графиком покупателей

---

**Задача 4.3.2.** Изучите, как средний интервал между заказами влияет на удержание клиентов.

- Рассчитайте среднее время между заказами для двух групп пользователей:
    - совершившие 2–4 заказа;
    - совершившие 5 и более заказов.
- Исследуйте, как средний интервал между заказами влияет на вероятность повторного заказа, и сделайте выводы.

---


In [ ]:
pr_f.query('is_two == True and is_five == False')['avg_days_between'].mean()

In [ ]:
pr_f.query('is_five == True')['avg_days_between'].mean()

Средний интервал между заказами у пользователей, совершивших более 5 заказов в два раза больше чем у пользователей с 2-4 заказами 

---

#### 4.4. Корреляционный анализ количества покупок и признаков пользователя

Изучите, какие характеристики первого заказа и профиля пользователя могут быть связаны с числом покупок. Для этого используйте универсальный коэффициент корреляции `phi_k`, который позволяет анализировать как числовые, так и категориальные признаки.

---

**Задача 4.4.1:** Проведите корреляционный анализ:
- Рассчитайте коэффициент корреляции `phi_k` между признаками профиля пользователя и числом заказов (`total_orders`). При необходимости используйте параметр `interval_cols` для определения интервальных данных.
- Проанализируйте полученные результаты. Если полученные значения будут близки к нулю, проверьте разброс данных в `total_orders`. Такое возможно, когда в данных преобладает одно значение: в таком случае корреляционный анализ может показать отсутствие связей. Чтобы этого избежать, выделите сегменты пользователей по полю `total_orders`, а затем повторите корреляционный анализ. Выделите такие сегменты:
    - 1 заказ;
    - от 2 до 4 заказов;
    - от 5 и выше.
- Визуализируйте результат корреляции с помощью тепловой карты.
- Ответьте на вопрос: какие признаки наиболее связаны с количеством заказов?

---

In [ ]:
matrix = pr_f[['first_order_date','last_order_date','first_device','first_region',
     'first_service','first_event_type','total_orders','avg_revenue_rub','avg_tickets','avg_days_between']].phik_matrix()

matrix = matrix.loc[matrix.index != 'total_orders'][['total_orders']].sort_values(by='total_orders', ascending=False)
matrix

In [ ]:
sns.heatmap(data=matrix,vmin=0,vmax=1, annot=True, fmt=".2f", cmap="coolwarm")

plt.title("Матрица корреляции количества заказов")

Из анализа корреляции количества заказов можно сделать вывод о том, что больше всего влияют "временные признаки" - 
first_order_date,last_order_date, avg_days_between

### 5. Общий вывод и рекомендации

В конце проекта напишите общий вывод и рекомендации: расскажите заказчику, на что нужно обратить внимание. В выводах кратко укажите:

- **Информацию о данных**, с которыми вы работали, и то, как они были подготовлены: например, расскажите о фильтрации данных, переводе тенге в рубли, фильтрации выбросов.
- **Основные результаты анализа.** Например, укажите:
    - Сколько пользователей в выборке? Как распределены пользователи по числу заказов? Какие ещё статистические показатели вы подсчитали важным во время изучения данных?
    - Какие признаки первого заказа связаны с возвратом пользователей?
    - Как связаны средняя выручка и количество билетов в заказе с вероятностью повторных покупок?
    - Какие временные характеристики влияют на удержание (день недели, интервалы между покупками)?
    - Какие характеристики первого заказа и профиля пользователя могут быть связаны с числом покупок согласно результатам корреляционного анализа?
- Дополните выводы информацией, которая покажется вам важной и интересной. Следите за общим объёмом выводов — они должны быть компактными и ёмкими.

В конце предложите заказчику рекомендации о том, как именно действовать в его ситуации. Например, укажите, на какие сегменты пользователей стоит обратить внимание в первую очередь, а какие нуждаются в дополнительных маркетинговых усилиях.

Для формирования датафрейма для анализа был выполнен SQL запрос из базы данных. В нем проверены типы данных и пропуски. В результате предварительной обработки были приведены валюту к единой (руб). Пропуски были лишь в столбце days_since_prev, обусловленные условиями задания. Данные были исследованы на аномальные занчения - найдены выбросы в выручке. Эти значения отфильтрованы по 99 перцентелю. Затем был создан датафрейм с профилем пользователя. В него добавлены два бинарных признках о количество заказов (больше 2 и больше 5). В профиле содержится 21700 пользователь. Также были найдены выбросы в количестве заказов, которые отфильтрованы по 95 перцентелю (9987 значений). Был проведено исселдование распределения по типу меропрития, устройтсву, региону и оператору -  по этим данным распределение примерно равномерное. Было изучено распределение по выручке - основыне значения находятся в диапазоне 500р. Пользователи были разбиты на сегменты и исседованы статистические показатели. Был проведен корреляционный анализ количества заказов - наибольшее влияние оказывают признаки с датой и временем. 

Сегмент пользователей с 5+ заказами является наиболее стабильным и приносит наибольшую выручку, поэтому стоит сконцентрироваться на удержании таких клиентов. При этом набиолее популряные мероприятия таких пользователей - концерты. Поскольку количество заказов коррелирует с "временными признаками", возможно стоит чаще предлагать пользователям акции или спецпредложенич, чтобы частота покупок была выше

### 6. Финализация проекта и публикация в Git

Когда вы закончите анализировать данные, оформите проект, а затем опубликуйте его.

Выполните следующие действия:

1. Создайте файл `.gitignore`. Добавьте в него все временные и чувствительные файлы, которые не должны попасть в репозиторий.
2. Сформируйте файл `requirements.txt`. Зафиксируйте все библиотеки, которые вы использовали в проекте.
3. Вынести все чувствительные данные (параметры подключения к базе) в `.env`файл.
4. Проверьте, что проект запускается и воспроизводим.
5. Загрузите проект в публичный репозиторий — например, на GitHub. Убедитесь, что все нужные файлы находятся в репозитории, исключая те, что в `.gitignore`. Ссылка на репозиторий понадобится для отправки проекта на проверку. Вставьте её в шаблон проекта в тетрадке Jupyter Notebook перед отправкой проекта на ревью.

**Вставьте ссылку на проект в этой ячейке тетрадки перед отправкой проекта на ревью.**